# Model of work distribution in a flat organization

* no managers; only workers
* no support staff; all infrastructure for communication and tasks works without failure
* no dark patterns

TODO:
* use realistic distributions (e.g., power law) instead of uniform
* instead of searching randomly, search second-order contacts

In [1]:
import random
import numpy
print("numpy", numpy.__version__)
import pandas
print("pandas", pandas.__version__)
import seaborn
print("seaborn", seaborn.__version__)
import matplotlib
print("matplotlib", matplotlib.__version__)
from matplotlib import pyplot as plt
import time
import sys
print(sys.version_info)

numpy 1.23.5
pandas 1.5.2
seaborn 0.12.2
matplotlib 3.6.2
sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)


In [ ]:
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

In [ ]:
random.seed(10)

In [ ]:
import lib_simulation

### global variables for simulation

#### user-defined

In [ ]:
import configuration_simCount25_skills3_levels3_duration1_people2to10_social0to8_ticks100 as config

### input validation

In [ ]:
if len(config.skill_set_for_people)<1:
    raise Exception("invalid value")
if len(config.skill_set_for_tasks)<1:
    raise Exception("invalid value")
    
if config.max_skill_level_per_person<1:
    raise Exception("invalid value")
if config.max_skill_level_per_task<1:
    raise Exception("invalid value")
if config.max_task_duration_in_ticks<1:
    raise Exception("invalid value")
    
if config.min_number_of_people<1:
    raise Exception("invalid value")
if config.max_number_of_people<1:
    raise Exception("invalid value")
if ((config.min_number_of_people) > (config.max_number_of_people)):
    print(config.min_number_of_people)
    print(config.max_number_of_people)
    print((config.min_number_of_people) > (config.max_number_of_people))
    raise Exception("invalid value")

if config.min_social_circle_size<0:
    raise Exception("invalid value")
if config.max_social_circle_size<0:
    raise Exception("invalid value")
if config.min_social_circle_size>config.max_social_circle_size:
    print(config.min_social_circle_size)
    print(config.max_social_circle_size)
    raise Exception("invalid value")
    
if config.max_ticks_to_simulate<1:
    raise Exception("invalid value")
    
if config.number_of_simultions<1:
    raise Exception("invalid value")

### Initialize Simulation

In [ ]:
list_of_dicts = []

max_task_duration = 5

start_time = time.time()
for number_of_people in range(config.min_number_of_people,config.max_number_of_people+1):
    print("number of people:",number_of_people,round(time.time()-start_time,2),"seconds")
    for social_circle_size in range(config.min_social_circle_size,config.max_social_circle_size+1):
        #print("  social circle size:",social_circle_size,round(time.time()-start_time,2),"seconds")
        for max_task_duration_in_ticks in range(1,max_task_duration+1):
            #print("    task duration:",max_task_duration_in_ticks,round(time.time()-start_time,2),"seconds")
            for sim_index in range(config.number_of_simultions+1):
                #print("      sim index:",sim_index,round(time.time()-start_time,2),"seconds")
                this_sim_dict = {'sim index': sim_index}

                list_of_people = []
                for person_index in range(number_of_people):
                    list_of_people.append(lib_simulation.CreatePerson(person_index, 
                                                                      config.skill_set_for_people, 
                                                                      config.max_skill_level_per_person))

                tasks_dict = {} # track the generated tasks for post-simulation analysis

                task_id = -1

                # randomly distribute tasks to people
                for index in range(len(list_of_people)):
                    #print('person ID',list_of_people[index].unique_id)
                    task_id+=1
                    tasks_dict = lib_simulation.new_task(task_id,
                                                         config.skill_set_for_tasks,
                                                         config.max_skill_level_per_task,
                                                         max_task_duration_in_ticks,
                                                         tasks_dict)
                    list_of_people[index].backlog_of_tasks.append(tasks_dict[task_id])


                list_of_people, tasks_dict = lib_simulation.simulate(False, # show narrative
                                                                     False, # work journal
                                                                     config.skill_set_for_tasks,
                                                                     config.max_skill_level_per_task,
                                                                     config.max_ticks_to_simulate,
                                                                     max_task_duration_in_ticks,
                                                                     list_of_people,
                                                                     tasks_dict)
    
                this_sim_dict['completed task count at tick 100'] = len(tasks_dict.keys())
                this_sim_dict['number of people'] =number_of_people
                this_sim_dict['number of people in contact list'] = social_circle_size
                this_sim_dict['task duration'] = max_task_duration_in_ticks
                this_sim_dict['aggregate person skills'] = lib_simulation.get_aggregate_person_dict(list_of_people,
                                                                                                    config.skill_set_for_people,
                                                                                                    config.max_skill_level_per_person)
                list_of_dicts.append(this_sim_dict)

## Post-simulation analysis

In [ ]:
df = pandas.DataFrame(list_of_dicts)
df.head()

In [ ]:
df.shape

In [ ]:
list_of_people[0].skill_specialization_dict

In [ ]:
df.columns

In [ ]:
df.max()

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.stripplot.html
seaborn.stripplot(df.loc[(df['number of people in contact list']==0) & (df['task duration']==1)], 
                  x='number of people', 
                  y='completed task count at tick 100',
                  jitter=True)
plt.title("social circle size 0\ntask duration = coordination duration");

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.swarmplot.html
seaborn.stripplot(df.loc[(df['number of people in contact list']==0) & (df['task duration']==max_task_duration)], 
                  x='number of people', 
                  y='completed task count at tick 100',
                  jitter=True)
plt.title("social circle size 0\ntask duration = "+str(max_task_duration)+"x coordination duration");

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.stripplot.html
seaborn.stripplot(df.loc[(df['number of people in contact list']==config.max_social_circle_size) 
                       & (df['task duration']==1)], 
                  x='number of people', 
                  y='completed task count at tick 100',
                  jitter=True)
plt.title("social circle size "+str(config.max_social_circle_size)+
          "\ntask duration = coordination duration");

In [ ]:
# TODO: 
# https://stackoverflow.com/questions/66974670/displaying-averages-graphically-on-seaborn-swarm-plots

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.swarmplot.html
seaborn.swarmplot(df.loc[(df['number of people']==config.max_number_of_people) 
                       & (df['task duration']==1)], 
                  x='number of people in contact list', 
                  y='completed task count at tick 100')
plt.title("organization comprised of "+str(config.max_number_of_people)+
          " people\ntask duration = coordination duration");

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.swarmplot.html
seaborn.swarmplot(df.loc[(df['number of people']==config.max_number_of_people) &
                         (df['task duration']==max_task_duration)], 
                  x='number of people in contact list', 
                  y='completed task count at tick 100')
plt.title("organization comprised of "+str(config.max_number_of_people)+
          " people\ntask duration = "+str(max_task_duration)+
          "x coordination duration");